# generate_regions_auto

this notebook's purpose is to automatically find regions of interest in the ocean current maps and slice them out

in other words, find the regions with actual data and get some info about the coordinates

then pickle it to use the region data in `access_thredds` notebook

### note

to generate a pickle for a different region, change the `dataset_url` variable and rerun the whole notebook

In [ ]:
%matplotlib inline

In [ ]:
import copy
import math
import pickle
import sys
import time
import numpy as np
import xarray as xr
from parcels import FieldSet
from datetime import timedelta

In [ ]:
dataset_url_6kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd"
dataset_url_2kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd"
dataset_url_1kmhourly = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/1km/hourly/RTV/HFRADAR_US_West_Coast_1km_Resolution_Hourly_RTV_best.ncd"
filename_dict = {
    dataset_url_1kmhourly: "west_coast_1km_hourly",
    dataset_url_2kmhourly: "west_coast_2km_hourly",
    dataset_url_6kmhourly: "west_coast_6km_hourly"
}

### CHANGE THIS TO CHANGE THE DATA RETRIEVED ###
dataset_url = dataset_url_2kmhourly
### ######################################## ###

time_chunk_size = 50
chunk_dict = {"time": time_chunk_size}
to_drop = ["dopx", "dopy", "hdop", "number_of_sites", "number_of_radials"]

In [ ]:
fs_xr = xr.open_dataset(dataset_url, chunks=chunk_dict, drop_variables=to_drop)
fs_xr

### sampling data at multiple times

for some hours of data, parts of the data aren't recorded or available to avoid forgetting about the lost data, we check multiple hours of data vs just using the latest data as reference

on the contrary, if some important radar went offline for a particularly long time, this won't work well

In [ ]:
timespan_to_sample = np.timedelta64(15,'h')
time_end = fs_xr["time"].max()
time_start = time_end - timespan_to_sample
fs_xr_slice = fs_xr.sel(time=slice(time_start, time_end))
fs_xr_slice

In [ ]:
lat_diff = float(fs_xr_slice["lat"].diff(dim="lat").max().values)
lon_diff = float(fs_xr_slice["lon"].diff(dim="lon").max().values)
print(lat_diff, lon_diff)

In [ ]:
# rounding the stuff is fine since xarray does nearest neighbor lookups
lat_range = (math.floor(fs_xr_slice["lat"].min()), math.ceil(fs_xr_slice["lat"].max()))
lon_range = (math.floor(fs_xr_slice["lon"].min()), math.ceil(fs_xr_slice["lon"].max()))
print(lat_range, lon_range)

## finding areas of interest

a quickly conceived method of continuously dividing the map smaller and smaller

the more i look at the code the more jank it becomes

the only reason this works is because generally there's only one region of data in a latitudal region

this shit takes ages to run so grab a drink (up to five minutes)

In [ ]:
# for adjacent regions found to be of interest, merge them together into one
def merge_regions(regs, diff, num_divs):
    merged = []
    i = 0
    while i < len(regs):
        start = regs[i]
        count = 1
        # region is as small as possible, don't bother merging
        # reached end of list, last element won't merge
        if start[2] or i >= len(regs) - 1:
            end = start
        else:
            while regs[i + 1][0] - regs[i][1] <= diff + sys.float_info.epsilon:
                count += 1
                i += 1
                # reached end of list, break
                if i == len(regs) - 1:
                    break
            end = regs[i]
        # the number of regions being merged equals the number of divisions
        # this means it cannot be divided further
        if start[2] or count == num_divs:
            merged.append((start[0], end[1], True))
        else:
            merged.append((start[0], end[1], False))
        i += 1
    print(merged)
    print("------------------------")
    return merged

In [ ]:
# number of regions to divide the map into
# higher values are better if the regions are closer together
lat_divs = 18
lon_divs = 7

In [ ]:
start = time.time()
# dividing latitude regions of interest
# regions have a flag as the 3rd element marking whether they are the smallest possible or not
# at the end of the algorithm, the flags won't necessarily be all True
lat_regions = [(lat_range[0], lat_range[1], False)]
lat_found = False
while not lat_found:
    prev_regions = lat_regions.copy()
    # clear regions to find new regions of interest
    lat_regions = []
    for r in prev_regions:
        if r[2]:
            lat_regions.append(r)
            continue
        # lat_divs + 1 or else we only get lat_divs - 1 regions
        r_secs = np.linspace(r[0], r[1], lat_divs + 1)
        for i in range(0, lat_divs):
            xr_slice = fs_xr_slice.sel(lat=slice(r_secs[i], r_secs[i + 1]), lon=slice(lon_range[0], lon_range[1]))
            # check if region actually has data
            # the main bottleneck, this check takes so long to run
            if not np.isnan(xr_slice["u"]).all().values:
                # append latitude values according to the coordinate values of the data
                lat_regions.append((float(xr_slice["lat"].min().values), float(xr_slice["lat"].max().values), False))
    lat_regions = merge_regions(lat_regions, lat_diff, lat_divs)
    # this check should be fine since it takes coordinates straight from the data itself
    if lat_regions == prev_regions:
        lat_found = True
end = time.time()
print(f"time elapsed: {end - start}")
lat_regions

HFR 6km 52.875375270843506 seconds

HFR 2km 163.83207821846008 seconds w/o lat lon chunking

In [ ]:
start = time.time()
# dividing each latitude region into longitude regions
# stored as a list of lists
lon_regions = [[(lon_range[0], lon_range[1], False)]] * len(lat_regions)
lon_found = False
while not lon_found:
    # deep copy list because it has lists inside
    prev_regions = copy.deepcopy(lon_regions)
    for s in range(0, len(lat_regions)):
        lon_regions[s] = []
        for r in prev_regions[s]:
            if r[2]:
                lon_regions[s].append(r)
                continue
            r_secs = np.linspace(r[0], r[1], lon_divs + 1)
            for i in range(0, lon_divs):
                xr_slice = fs_xr_slice.sel(lat=slice(lat_regions[s][0], lat_regions[s][1]), lon=slice(r_secs[i], r_secs[i + 1]))
                if not np.isnan(xr_slice["u"]).all().values:
                    lon_regions[s].append((float(xr_slice["lon"].min().values), float(xr_slice["lon"].max().values), False))
    for i in range(0, len(lon_regions)):
        lon_regions[i] = merge_regions(lon_regions[i], lon_diff, lon_divs)
    if lon_regions == prev_regions:
        lon_found = True
end = time.time()
print(f"time elapsed: {end - start}")
lon_regions

### the pickle dump

where the data does dumps

In [ ]:
regs = []
for i in range(0, len(lat_regions)):
    for lonr in lon_regions[i]:
        regs.append((lat_regions[i], lonr))
d = filename_dict[dataset_url]
with open(f"regions_{d}.p", "wb") as f:
    pickle.dump(regs, f)

### plots of regions found

mostly for debugging purposes

In [ ]:
# show a plot of all the regions of interest found
for i in range(0, len(lat_regions)):
    for lonr in lon_regions[i]:
        xarr_sliced = fs_xr_slice.sel(lat=slice(lat_regions[i][0], lat_regions[i][1]), lon=slice(lonr[0], lonr[1]))
        print((math.floor(xarr_sliced["lat"].min()), math.ceil(xarr_sliced["lat"].max())))
    #     print(xarr_sliced["u"].sum())
        a = xarr_sliced["u"].sum().values
        print(f"sum U: {a}")
        fs_sliced = FieldSet.from_xarray_dataset(xarr_sliced, dict(U="u",V="v"), dict(lat="lat",lon="lon",time="time"))
        fs_sliced.U.show(vmin=-1, vmax=1)